In [16]:
from dotenv import load_dotenv

_ = load_dotenv()

In [30]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [31]:
tool = TavilySearchResults(max_results=2)

In [33]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

## checkpointing example

In [64]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [65]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [66]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o", streaming=True)
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [67]:
messages = [HumanMessage(content="What is the weather in sf?")]

## Thread for session state

In [73]:
thread = {"configurable": {"thread_id": "1"}}

In [74]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Los Angeles is currently warmer than San Francisco. The temperature in Los Angeles is approximately 10.6°C (51.1°F), while in San Francisco, it is 8.3°C (46.9°F).', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf'}, id='run-3248ed5c-fbe7-4256-8b47-ccc1b17bb066-0')]}


In [75]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Currently, Los Angeles is experiencing clear weather with a temperature of approximately 10.6°C (51.1°F), which is warmer than San Francisco's temperature of 8.3°C (46.9°F). If there's anything else you need to know about Los Angeles, feel free to ask!", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf'}, id='run-072cc91e-09b3-4c1d-ba43-34d2261fd321-0')]}


In [76]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Los Angeles is currently warmer than San Francisco, with a temperature of approximately 10.6°C (51.1°F) compared to San Francisco's 8.3°C (46.9°F).", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf'}, id='run-485fbb07-206f-4057-9ee1-5ee0cb23062f-0')]}


## adding new thread

In [77]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Could you please clarify what you're comparing? Are you asking about the temperature between two specific locations, weather conditions, clothing items, or something else?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf'}, id='run-80a45cd3-5931-4747-a174-d90d9c518480-0')]}


## Token level Streaming

In [80]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [81]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_QOdARn4s2ee9z1CzPQYrnh2U', 'type': 'tool_call'}
Back to the model!
The| current| weather| in| San| Francisco| is| clear| with| a| temperature| of| |8|.|3|°C| (|46|.|9|°F|).| The| wind| is| coming| from| the| northwest| at| |11|.|0| mph| (|17|.|6| k|ph|),| and| the| humidity| is| |74|%.| The| visibility| is| good| at| |16|.|0| km| (|9|.|0| miles|).| There| is| no| precipitation|,| and| the| atmospheric| pressure| is| |102|6|.|0| mb| (|30|.|3| in|).|